# Problem 1: Classical Planning 

In this assignment, we will learn how to define a planning model in the Planning Domain Definition Language (PDDL) in the elevator environment. PDDL is a language used to describe planning problems and is widely used in artificial intelligence for automated planning.

You can understand the elevator domain by the `0_env_walkthrough.ipynb` notebook.

In PDDL, we separate the language to two files:

1. `domain.pddl`: This file defines the general structure of the domain/environment, including object types, states, and actions, which can be used to solve any specific problem within this domain. This is **Task 1**.
2. `problem.pddl`: This file defines a specific instance of the task, including the objects in the particular task, the initial state, and the goal state. This is **Task 2**.

## 1. Defining the Domain File

In PDDL, the `domain` file defines object types (`types`), states (`predicates`), and actions (`actions`). The domain file is independent of specific tasks, which means it describes the general structure of the domain. 

### 1.1 Types

Types define the categories of objects in our domain:

- `level`: Represents the floors in the building.
- `person`: Represents the people who need to be transported by the elevator.

### 1.2 Predicates

Predicates represent the states in our domain. They describe properties of the world or relationships between objects. For our elevator domain, we will define the following predicates:

- `elevator_at(?l - level)`: The elevator is at a specific level.
- `person_at(?p - person, ?l - level)`: A person is at a specific level.
- `person_in_elevator(?p - person)`: A person is in the elevator.
- `elevator_empty()`: The elevator is empty.
- `door_open(?l - level)`: The door is open at a specific level.
- `adjacent_up(?from - level, ?to - level)`: Indicates that ?to is the level above ?from.
- `adjacent_down(?from - level, ?to - level)`: Indicates that ?to is the level below ?from.

### 1.3 Actions

Next, we define the actions in the elevator domain, as shown in the *walkthrough* notebook, there are six actions in this domain:

1. `open_door(?l - level)`: Opens the elevator door at a specific level.
2. `close_door(?l - level)`: Closes the elevator door at a specific level.
3. `load(?p - person, ?l - level)`: Picks up a person when the door is open, and the elevator is empty.
4. `unload(?p - person, ?l - level)`: Drops off a person when the door is open.
5. `move_up(?from - level, ?to - level)`: Moves the elevator up one level.
6. `move_down(?from - level, ?to - level)`: Moves the elevator down one level.

### Task 1: Complete the Planning Domain Definition

In this task, we will write the PDDL code for the elevator domain as a Python string. You need to complete the PDDL code by replacing the `___` with the appropriate PDDL syntax. After finishing, the domain will be stored in a file named `elevator_domain.pddl` for the subsequent tasks.

In [1]:
# COPY-FLAG-1-START

pddl_domain = """
(define (domain elevator)
  (:types 
    level person ; Define types for levels and persons
  )
  
  (:predicates
    (elevator_at ?l - level) ; The elevator is at a specific level
    (person_at ?p - person ?l - level) ; A person is at a specific level
    (person_in_elevator ?p - person) ; A person is in the elevator
    (elevator_empty) ; The elevator is empty
    (door_open ?l - level) ; The door is open at a specific level
    (adjacent_up ?from ?to - level) ; Defines that ?to is the level above ?from
    (adjacent_down ?from ?to - level) ; Defines that ?to is the level below ?from
  )
  
  ; move_up: The elevator can only move up one level
  (:action move_up
    :parameters (?from ?to - level)
    :precondition (and (elevator_at ?from) ;
                      (adjacent_up ?from ?to) ;
                      (elevator_empty) ;
                      (not (door_open ?from))) ; FILL IN the precondition for move_up
    :effect (and (not(elevator_at ?from)) ;
                (elevator_at ?to)) ; FILL IN the effect for move_up
  )

  ; move_down: The elevator can only move down one level
  (:action move_down
    :parameters (?from ?to - level)
    :precondition (and (elevator_at ?from) ;
                      (adjacent_down ?from ?to) ;
                      (not (door_open ?from))) ; FILL IN the precondition for move_down
    :effect (and (not(elevator_at ?from)) ;
                (elevator_at ?to)) ; FILL IN the effect for move_down
  )

  ; open_door: Open the door without considering picking up people
  (:action open_door
    :parameters (?l - level)
    :precondition (and (elevator_at ?l) ;
                      (not (door_open ?l))) ; FILL IN the precondition for open_door
    :effect (and (door_open ?l)) ; FILL IN the effect for open_door
  )

  ; close_door: Close the door
  (:action close_door
    :parameters (?l - level)
    :precondition (and (elevator_at ?l) ;
                      (door_open ?l)) ; FILL IN the precondition for close_door
    :effect (and (not (door_open ?l))) ; FILL IN the effect for close_door
  )

  ; load: Pick up a person, requires the door to be open and the elevator to be empty
  (:action load
    :parameters (?p - person ?l - level)
    :precondition (and (elevator_at ?l) ;
                      (door_open ?l) ;
                      (elevator_empty) ;
                      (person_at ?p ?l)) ; FILL IN the precondition for load
    :effect (and (not(elevator_empty)) ;
                  (not(person_at ?p ?l)) ;
                  (person_in_elevator ?p)) ; FILL IN the effect for load
  )

  ; unload: Drop off a person, requires the door to be open and the person to be in the elevator
  (:action unload
    :parameters (?p - person ?l - level)
    :precondition (and (elevator_at ?l) ;
                      (door_open ?l) ;
                      (not(elevator_empty)) ;
                      (person_in_elevator ?p)) ; FILL IN the precondition for unload
    :effect (and (not(person_in_elevator ?p));
                  (elevator_empty) ;
                  (person_at ?p ?l)) ; FILL IN the effect for unload
  )
)
"""

# COPY-FLAG-1-END

with open("elevator_domain.pddl", "w") as file:
    file.write(pddl_domain)

## 2. Defining the Problem File

The planning`problem` file defines a specific instance of the environment, including the objects involved, the initial state, and the goal state. We will use the `generate_pddl_from_config` function to generate a PDDL file for a specific elevator planning problem, which can be widely utilized in different settings. The function takes a configuration array, where each element represents a level, and a value of `1` indicates the presence of a person at that level.

### Task 2: Generate the Planning Problem Definition

In this task, you need to complete a Python function that generates a PDDL file for an elevator planning problem. The PDDL file should describe the initial state of the elevator and the people at different levels, as well as the goal state where all people are at the ground level (`level0`).

The function takes a configuration array that specifies which levels have people and saves the generated PDDL file to a specified path.

An example for a configuration array is `[0, 0, 1]` which indicates that there is one person at level 2.


In [2]:
import numpy as np


# COPY-FLAG-2-START

def generate_pddl_from_config(config_map, output_file):
    num_levels = len(config_map)
    persons = [f"person{i + 1}" for i in range(np.sum(config_map))]

    # PDDL header
    pddl = "(define (problem elevator_problem)\n"
    pddl += "  (:domain elevator)\n"

    # Objects
    levels = " ".join([f"level{i}" for i in range(num_levels)])
    persons_str = " ".join(persons)
    pddl += f"  (:objects\n    {levels} - level\n    {persons_str} - person\n  )\n\n"

    # Initial state
    pddl += "  (:init\n"
    pddl += "    (elevator_at level0)\n"  # Assuming elevator starts at level 0

    person_idx = 0
    for level, has_person in enumerate(config_map):
        if has_person:
            person = persons[person_idx]
            pddl += f"    (person_at {person} level{level})\n"
            person_idx += 1

    # Add elevator states and adjacencies
    pddl += "    (elevator_empty)\n"
    for i in range(num_levels - 1):
        pddl += f"    (adjacent_up level{i} level{i+1})\n"  # Fill in this line to define adjacency up
        pddl += f"    (adjacent_down level{i+1} level{i})\n"  # Fill in this line to define adjacency down

    pddl += "  )\n\n"

    # Goal state
    pddl += "  (:goal\n"
    pddl += "    (and\n"
    for person in persons:
        pddl += f"      (person_at {person} level0)\n"  # Fill in this line to define the goal for each person
    pddl += "    )\n"
    pddl += "  )\n"
    pddl += ")"

    # Save PDDL to file
    with open(output_file, 'w') as file:
        file.write(pddl)

    return pddl

# COPY-FLAG-2-END

Then generate the problem PDDL file for the following two configurations:

In [3]:
# Example1 usage: 3 levels, 1 person at level 2
config_1 = np.array([0, 0, 1], dtype=int)
pddl_string_1 = generate_pddl_from_config(config_1, "elevator_problem1.pddl")

# Example2 usage: 5 levels, 1 person at level 1, 3, 4
config_2 = np.array([0, 1, 0, 1, 1], dtype=int)
pddl_string_2 = generate_pddl_from_config(config_2, "elevator_problem2.pddl")

## 3. Testing and Verification

Once you have written your `domain` and `problem` files, you have successfully modeled the elevator problem in PDDL, next, we will use a **PDDL planner** to solve the modelled problem. The planner will generate a sequence of actions to achieve the goal state from the initial state. You can verify this by using the code below, which allows you to read the generated `domain.pddl` and `problem.pddl` files and output the plan.

**Please Note**: If you cannot find a plan, you should check your PDDL code for errors. *A well-defined PDDL model should always have a valid plan.*

In [4]:
from EleEnv.PDDL import Planner
import sys

domain = 'elevator_domain.pddl'
problems = ['elevator_problem1.pddl', 'elevator_problem2.pddl']

for problem in problems:
    planner = Planner()
    plan = planner.solve(domain, problem)
    verbose = True
    if plan is not None:
        print('plan:')
        for act in plan:
            print(act if verbose else act.name + ' ' + ' '.join(act.parameters))
    else:
        sys.exit('No plan was found')

    print('Plan for {} is done\n-------------\n'.format(problem))

plan:
action: move_up
  parameters: ['level0', 'level1']
  positive_preconditions: [['adjacent_up', 'level0', 'level1'], ['elevator_at', 'level0'], ['elevator_empty']]
  negative_preconditions: [['door_open', 'level0']]
  add_effects: [['elevator_at', 'level1']]
  del_effects: [['elevator_at', 'level0']]

action: move_up
  parameters: ['level1', 'level2']
  positive_preconditions: [['elevator_at', 'level1'], ['elevator_empty'], ['adjacent_up', 'level1', 'level2']]
  negative_preconditions: [['door_open', 'level1']]
  add_effects: [['elevator_at', 'level2']]
  del_effects: [['elevator_at', 'level1']]

action: open_door
  parameters: ['level2']
  positive_preconditions: [['elevator_at', 'level2']]
  negative_preconditions: [['door_open', 'level2']]
  add_effects: [['door_open', 'level2']]
  del_effects: []

action: load
  parameters: ['person1', 'level2']
  positive_preconditions: [['person_at', 'person1', 'level2'], ['door_open', 'level2'], ['elevator_at', 'level2'], ['elevator_empty']]